In [ ]:
# reading in data and printing first 1000 chars
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
print("length of dataset in characters: ", len(text))
print(text[:1000])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

In [8]:
# getting all unique chars and determining possible output classes- vocab_size

chars = list(sorted(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
# creating an encoding and decoding to convert string to int and vice versa

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for ch,i in stoi.items()}

encode = lambda s: [stoi[ch] for ch in s]
decode = lambda i: ''.join([itos[chi] for chi in i])

print(encode("hii there"))
print(decode(encode("hii there")))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
import torch


data = torch.tensor(encode(text), dtype=torch.long) # using long type for class ints
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:]

In [25]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # block_size is like context window

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,)) # list of len(batch_size) of ints in data
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')    

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [ ]:
# starting with a bigram without training.

import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class bigram(nn.Module):
    def __init__(self, vocab_size, n_embd):
        super().__init__()
        self.n_embd_table = nn.Embedding(vocab_size, n_embd)
        
    def forward(self, IX, targets=None): # ix is (B,T) with int representing row to be queried for embedding.
        logits = self.n_embd_table(IX) # (B, T, C)
        
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T) 
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None
        
        return logits, loss
    
    def generate(self, idx, max_new_token):
        for _ in range(max_new_token):
        
            logits, loss = self(idx)
            logits = logits[:,-1,:] # getting the last prediction. ideally this should be context length i think. 
            probs = F.softmax(logits, -1)
            pred = torch.multinomial(probs,1)
            idx = torch.cat((idx, pred),dim=1)
        
        return idx
            

model = bigram(vocab_size, vocab_size)
logits, loss = model(xb, yb)    
print(loss) # ln(1.0/65)=4.17 because vocab_size is 65. 65 possible outputs.
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_token=100)[0].tolist()))


tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [52]:
# starting with a bigram without training.

import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class bigram(nn.Module):
    def __init__(self, vocab_size, n_embd):
        super().__init__()
        self.n_embd_table = nn.Embedding(vocab_size, n_embd)
        
    def forward(self, IX, targets=None): # ix is (B,T) with int representing row to be queried for embedding.
        logits = self.n_embd_table(IX) # (B, T, C)
        
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T) 
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None
        
        return logits, loss
    
    def generate(self, idx, max_new_token):
        for _ in range(max_new_token):
        
            logits, loss = self(idx)
            logits = logits[:,-1,:] # getting the last prediction. ideally this should be context length i think. 
            probs = F.softmax(logits, -1)
            pred = torch.multinomial(probs,1)
            idx = torch.cat((idx, pred),dim=1)
        
        return idx
            
model = bigram(vocab_size, vocab_size)
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-3)


training_step = 10000
batch_size = 32

for step in range(training_step):
    
    xb, yb = get_batch('train') # global batch_size being used here
    # print(f"input shape is {xb.shape} and target shape is {yb.shape}")
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())    
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_token=1000)[0].tolist()))


2.394822597503662

Ke sonidin y y a!
Caril.n y, denjxhece w illd CHAL, mer thoun s's:Conchuntilalllevise sthat dy hangilyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z,
Thitertho s?
NDan'spererfo cist ripl chys er orlese;
Yo jehof h hecere ek? wferommot mowo soaf yoit, ince his, t, f at. fal whetrimy bupof tor atha Bu!
JOutho f cimimave.
NEDUSt cir selle p wie wede
Ro n apenor f'Y tover witys an sh d w t e w!
CEOntiretoaveE IINpe, theck. cung.
ORIsthies hacin benqurd bll, d a r w wistatsowor ath
Fivet bloll ang a-I theeancu,
LINCI'T:
Sarry t 

In [55]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [80]:
xbow= torch.zeros_like(x)

for each_exm in range(x.shape[0]):
    for char_index in range(x.shape[1]):
        xprev = x[each_exm, :char_index+1] # (t, C) here batch gets eliminated since its 1d
        xbow[each_exm,char_index]=torch.mean(xprev, 0)  

xbow.shape   

torch.Size([4, 8, 2])

In [89]:
# tril = torch.tril(torch.ones(T, T))
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei
# tril.shape
# (tril @ x).shape

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [85]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [69]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape


# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)
